In [1]:
# importing the required libraries
import xarray as xr
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os
import pickle as pk
import pandas as pd

# set the path to the project
path='~/ForceSMIP/'

In [4]:
def train_random_forest(model,variable,ilat,ilon):
    '''
    This function trains a random forest model for a given model, variable, latitude and longitude. 
    Considers the PCA components (features) and the forced signal (targets).
    '''
    # features from PCA
    features=np.load(path+'PCA_data/'+model+'_'+variable+'_'+'features.npy')
    features=features.reshape(len(features), 10)
    # load the times, which say which timeslice each feature set corresponds to 
    times=np.load(path+'PCA_data/'+model+'_'+variable+'_'+'times.npy')

    # forced component (average across all realizations) for the given model, variable, latitude and longitude
    forced_dataset=xr.open_dataset(path+'forced_signals/'+model+'_'+variable+'.nc',decode_times=False).fillna(0).isel(lat=ilat,lon=ilon)
    if variable=='monmintasmin':
        forced_dataset=forced_dataset['tasmin'].values.flatten()
    if variable=='monmaxtasmax':
        forced_dataset=forced_dataset['tasmax'].values.flatten()
    if variable=='monmaxpr':
        forced_dataset=forced_dataset['pr'].values.flatten()
    else:
        forced_dataset=forced_dataset[variable].values.flatten()
    
    # selects only the first four components of the PCA, due to resource limitations
    updated_features=[]
    targets=[]
    for i in range(len(times)):
      updated_features.append(features[i][:4])
      targets.append(forced_dataset[times[i]])

    # RF model uses suboptimal hyperparameters for computational efficiency
    rf = RandomForestRegressor(n_estimators=35, max_depth=8, max_features=0.5, n_jobs=-1, random_state=0)
    rf.fit(updated_features, targets)
    return rf

def estimate_forced_component(model,variable,ds_list):
    '''
        This function estimates the forced component for a given model, variable and list of datasets.
        The ds_list is simply the list of the xarray Datasets from the corresponding Tier 1 evaluation.
    '''
    # the lag time in this case is 5 years (multiplied by 12 months), but could be expanded for increased accuracy
    lag_time=5*12
    # load the PCA model
    if variable=='monmintasmin':
        pca_reloaded = pk.load(open(path+'PCA_data/'+model+'_'+'tasmin'+'_pca.pkl','rb'))
    if variable=='monmaxtasmax':
        pca_reloaded = pk.load(open(path+'PCA_data/'+model+'_'+'tasmax'+'_pca.pkl','rb'))
    if variable=='monmaxpr':
        pca_reloaded = pk.load(open(path+'PCA_data/'+model+'_'+'pr'+'_pca.pkl','rb'))
    else:
        pca_reloaded = pk.load(open(path+'PCA_data/'+model+'_'+variable+'_pca.pkl','rb'))
    
    # calculate the PCA components for each dataset
    features_evaluations=[]
    for ds in ds_list:
        inputs=[]
        for i in range(0,len(ds.time)-lag_time):
            inputs.append(ds[variable].values[i:i+lag_time].flatten())
        features_evaluations.append(pca_reloaded.transform(inputs))
    
    # train the random forest model for each latitude and longitude, and predict the forced component
    forced_estimates=np.empty((len(ds_list),72,144,len(ds.time)-lag_time))
    for i in range(72):
        for j in range(144):
            rf=train_random_forest(model,variable,i,j)
            for k in range(len(ds_list)):
                forced_estimates[k][i][j] = rf.predict(features_evaluations[k].T[:4].T)
    return forced_estimates

def final_estimates(variable):
    '''
        Aggregates the estimates from each trained model and saves the final estimates to a netcdf file.
    '''
    ds_list=[]
    for model in ['CanESM5', 'CESM2', 'MIROC-ES2L', 'MIROC6', 'MPI-ESM1-2-LR']:
        ds=xr.DataArray(np.load(path+'estimates/'+model+'_'+variable+'.npy'),
                dims=('member', 'lat', 'lon', 'time'),
                coords={'member': ['A','B','C','D','E','F','G','H','I','J'],
                    'lat': np.arange(-90,90,2.5),
                    'lon': np.arange(-180,180,2.5),
                    'time': pd.date_range(start='1955-01-01', end='2022-12-31', freq='M')})
        ds_list.append(ds)
    ds_total=xr.concat(ds_list, dim='model').mean('model')

    i=0
    for member in ds_total.member.values.flatten():
        ds_total.isel(member=i).to_netcdf('final_estimates/'+variable+'_'+member+'_Tier1_RandomForest_Cropper.nc')
        i+=1

In [ ]:
var='tas'
ds_listed=[
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1A.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1B.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1C.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1D.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1E.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1F.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1G.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1H.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1I.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1J.195001-202212.nc',decode_times=False).fillna(0),
]
cesm2_forced=estimate_forced_component('CESM2',var,ds_listed)
np.save(path+'estimates/CESM2_'+var+'.npy',cesm2_forced)
canesm5_forced=estimate_forced_component('CanESM5',var,ds_listed)
np.save(path+'estimates/CanESM5_'+var+'.npy',canesm5_forced)
miroc6_forced=estimate_forced_component('MIROC6',var,ds_listed)
np.save(path+'estimates/MIROC6_'+var+'.npy',miroc6_forced)
miroc_es2l_forced=estimate_forced_component('MIROC-ES2L',var,ds_listed)
np.save(path+'estimates/MIROC-ES2L_'+var+'.npy',miroc_es2l_forced)
mpi_esm1_2_lr_forced=estimate_forced_component('MPI-ESM1-2-LR',var,ds_listed)
np.save(path+'estimates/MPI-ESM1-2-LR_'+var+'.npy',miroc6_forced)

In [ ]:
var='pr'
ds_listed=[
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1A.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1B.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1C.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1D.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1E.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1F.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1G.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1H.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1I.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1J.195001-202212.nc',decode_times=False).fillna(0),
]
cesm2_forced=estimate_forced_component('CESM2',var,ds_listed)
np.save(path+'estimates/CESM2_'+var+'.npy',cesm2_forced)
canesm5_forced=estimate_forced_component('CanESM5',var,ds_listed)
np.save(path+'estimates/CanESM5_'+var+'.npy',canesm5_forced)
miroc6_forced=estimate_forced_component('MIROC6',var,ds_listed)
np.save(path+'estimates/MIROC6_'+var+'.npy',miroc6_forced)
miroc_es2l_forced=estimate_forced_component('MIROC-ES2L',var,ds_listed)
np.save(path+'estimates/MIROC-ES2L_'+var+'.npy',miroc_es2l_forced)
mpi_esm1_2_lr_forced=estimate_forced_component('MPI-ESM1-2-LR',var,ds_listed)
np.save(path+'estimates/MPI-ESM1-2-LR_'+var+'.npy',miroc6_forced)

In [ ]:
var='psl'
ds_listed=[
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1A.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1B.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1C.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1D.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1E.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1F.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1G.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1H.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1I.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1J.195001-202212.nc',decode_times=False).fillna(0),
]
cesm2_forced=estimate_forced_component('CESM2',var,ds_listed)
np.save(path+'estimates/CESM2_'+var+'.npy',cesm2_forced)
canesm5_forced=estimate_forced_component('CanESM5',var,ds_listed)
np.save(path+'estimates/CanESM5_'+var+'.npy',canesm5_forced)
miroc6_forced=estimate_forced_component('MIROC6',var,ds_listed)
np.save(path+'estimates/MIROC6_'+var+'.npy',miroc6_forced)
miroc_es2l_forced=estimate_forced_component('MIROC-ES2L',var,ds_listed)
np.save(path+'estimates/MIROC-ES2L_'+var+'.npy',miroc_es2l_forced)
mpi_esm1_2_lr_forced=estimate_forced_component('MPI-ESM1-2-LR',var,ds_listed)
np.save(path+'estimates/MPI-ESM1-2-LR_'+var+'.npy',miroc6_forced)

In [ ]:
var='tos'
ds_listed=[
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1A.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1B.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1C.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1D.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1E.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1F.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1G.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1H.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1I.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1J.195001-202212.nc',decode_times=False).fillna(0),
]
cesm2_forced=estimate_forced_component('CESM2',var,ds_listed)
np.save(path+'estimates/CESM2_'+var+'.npy',cesm2_forced)
canesm5_forced=estimate_forced_component('CanESM5',var,ds_listed)
np.save(path+'estimates/CanESM5_'+var+'.npy',canesm5_forced)
miroc6_forced=estimate_forced_component('MIROC6',var,ds_listed)
np.save(path+'estimates/MIROC6_'+var+'.npy',miroc6_forced)
miroc_es2l_forced=estimate_forced_component('MIROC-ES2L',var,ds_listed)
np.save(path+'estimates/MIROC-ES2L_'+var+'.npy',miroc_es2l_forced)
mpi_esm1_2_lr_forced=estimate_forced_component('MPI-ESM1-2-LR',var,ds_listed)
np.save(path+'estimates/MPI-ESM1-2-LR_'+var+'.npy',miroc6_forced)

In [ ]:
var='monmaxtasmax'
ds_listed=[
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1A.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1B.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1C.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1D.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1E.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1F.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1G.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1H.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1I.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1J.195001-202212.nc',decode_times=False).fillna(0),
]
cesm2_forced=estimate_forced_component('CESM2',var,ds_listed)
np.save(path+'estimates/CESM2_'+var+'.npy',cesm2_forced)
canesm5_forced=estimate_forced_component('CanESM5',var,ds_listed)
np.save(path+'estimates/CanESM5_'+var+'.npy',canesm5_forced)
miroc6_forced=estimate_forced_component('MIROC6',var,ds_listed)
np.save(path+'estimates/MIROC6_'+var+'.npy',miroc6_forced)
miroc_es2l_forced=estimate_forced_component('MIROC-ES2L',var,ds_listed)
np.save(path+'estimates/MIROC-ES2L_'+var+'.npy',miroc_es2l_forced)
mpi_esm1_2_lr_forced=estimate_forced_component('MPI-ESM1-2-LR',var,ds_listed)
np.save(path+'estimates/MPI-ESM1-2-LR_'+var+'.npy',miroc6_forced)

In [ ]:
var='monmintasmin'
ds_listed=[
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1A.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1B.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1C.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1D.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1E.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1F.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1G.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1H.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1I.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1J.195001-202212.nc',decode_times=False).fillna(0),
]
cesm2_forced=estimate_forced_component('CESM2',var,ds_listed)
np.save(path+'estimates/CESM2_'+var+'.npy',cesm2_forced)
canesm5_forced=estimate_forced_component('CanESM5',var,ds_listed)
np.save(path+'estimates/CanESM5_'+var+'.npy',canesm5_forced)
miroc6_forced=estimate_forced_component('MIROC6',var,ds_listed)
np.save(path+'estimates/MIROC6_'+var+'.npy',miroc6_forced)
miroc_es2l_forced=estimate_forced_component('MIROC-ES2L',var,ds_listed)
np.save(path+'estimates/MIROC-ES2L_'+var+'.npy',miroc_es2l_forced)
mpi_esm1_2_lr_forced=estimate_forced_component('MPI-ESM1-2-LR',var,ds_listed)
np.save(path+'estimates/MPI-ESM1-2-LR_'+var+'.npy',miroc6_forced)

In [ ]:
var='monmaxpr'
ds_listed=[
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1A.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1B.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1C.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1D.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1E.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1F.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1G.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1H.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1I.195001-202212.nc',decode_times=False).fillna(0),
    xr.open_dataset(path+'Evaluation-Tier1/'+var+'_mon_1J.195001-202212.nc',decode_times=False).fillna(0),
]
cesm2_forced=estimate_forced_component('CESM2',var,ds_listed)
np.save(path+'estimates/CESM2_'+var+'.npy',cesm2_forced)
canesm5_forced=estimate_forced_component('CanESM5',var,ds_listed)
np.save(path+'estimates/CanESM5_'+var+'.npy',canesm5_forced)
miroc6_forced=estimate_forced_component('MIROC6',var,ds_listed)
np.save(path+'estimates/MIROC6_'+var+'.npy',miroc6_forced)
miroc_es2l_forced=estimate_forced_component('MIROC-ES2L',var,ds_listed)
np.save(path+'estimates/MIROC-ES2L_'+var+'.npy',miroc_es2l_forced)
mpi_esm1_2_lr_forced=estimate_forced_component('MPI-ESM1-2-LR',var,ds_listed)
np.save(path+'estimates/MPI-ESM1-2-LR_'+var+'.npy',miroc6_forced)

In [ ]:
for variable in ['tas', 'pr', 'psl', 'tos', 'monmintasmin', 'monmaxtasmax', 'monmaxpr']:
    final_estimates(variable)